# Read in required packages

In [1]:
import sys

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import scipy

#import scvi
import hashlib

from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
import seaborn as sns
import anndata
import os
import gc
import warnings

In [2]:
pwd

'/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object'

# Functions

In [3]:
# Code provided by Vitalii

def read_10x_output(smp_list, metadata=None, type = 'raw'):
    import os
    
    #Writing output from separate samples, processed using CellRanger, into a dictionary of Scanpy objects:
    ad = {}

    #Generate AnnData for each sample
    for sample_name in smp_list:
        path = sample_name
        for i in os.listdir(path):
            if type in i and 'h5' in i:
                file = f"{sample_name}_knee_FPR_0.01_filtered.h5"
                ad[sample_name] = sc.read_10x_h5(sample_name +'/'+file)
                ad[sample_name].var['ENSEMBL'] = ad[sample_name].var.index
                ad[sample_name].var.rename(columns = {'gene_ids':'SYMBOL'}, inplace = True)
                ad[sample_name].var.index = ad[sample_name].var['SYMBOL']
                ad[sample_name].var_names_make_unique() 
                ad[sample_name].obs['sample_id'] = sample_name
                ad[sample_name].obs['barcode'] = ad[sample_name].obs_names
                ad[sample_name].obs_names = ad[sample_name].obs['barcode']+"_"+ad[sample_name].obs['sample_id']

    #Merge AnnData objects from all the samples together    
    from scipy.sparse import vstack
    stack = vstack([ad[x].X for x in smp_list]) # stack data
    adata = sc.AnnData(stack, var = ad[smp_list[0]].var)
    adata.obs = pd.concat([ad[x].obs for x in smp_list], axis = 0)
    
    for c in adata.obs.columns:
        adata.obs[c] = adata.obs[c].astype('str')
    adata.obs =  adata.obs.copy()

    return adata, ad

In [4]:
metadata= pd.read_csv("/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object/metadata/metadata_harmonised.csv")

In [8]:
cd /lustre/scratch126/cellgen/team205/sharedData/kt21/all_multiome_cellbender

/lustre/scratch126/cellgen/team205/sharedData/kt21/all_multiome_cellbender


In [40]:
metadata

,Organ,Run_ID,Age,Anatomical_Site,Donor_ID,Technology,iRODs path,Farm Path,CellBender_Output_Path,Demultiplexed_Barcodes_Location
0,Skull,HCA_BN_F12627470_and_HCA_BN_F12605339,"5.8, 7.6, 8.3",knee,multiplexed,10X_multiome,/seq/illumina/cellranger-arc/cellranger-arc201...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team205/sharedData/...,NaN
1,Skull,HCA_BN_F12627471_and_HCA_BN_F12605340,"7.6, 8.3",hip,multiplexed,10X_multiome,/seq/illumina/cellranger-arc/cellranger-arc201...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team205/sharedData/...,NaN
2,Skull,HCA_BN_F12627472_and_HCA_BN_F12605341,"5.8, 8.3",hip/shoulder,multiplexed,10X_multiome,/seq/illumina/cellranger-arc/cellranger-arc201...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team205/sharedData/...,NaN
3,Skull,HCA_BN_F12627473_and_HCA_BN_F12605342,"7.6, 8.3",shoulder,multiplexed,10X_multiome,/seq/illumina/cellranger-arc/cellranger-arc201...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team205/sharedData/...,NaN
4,Skull,HCA_BN_F12874040_and_HCA_BN_F12865672,9.4,hip,2384,10X_multiome,/seq/illumina/cellranger-arc/cellranger-arc201...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team205/sharedData/...,NaN
...,...,...,...,...,...,...,...,...,...,...
127,Spine,41865_SB_200532_11018320,17,cervical,HDBR15948,10X_scRNA-seq,/seq/illumina/runs/41/41865/cellranger/cellran...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,NaN
128,Spine,41865_SB_200532_11018321,17,thoracic,HDBR15948,10X_scRNA-seq,/seq/illumina/runs/41/41865/cellranger/cellran...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,NaN
129,Spine,41865_SB_200532_11018322,17,thoracic,HDBR15948,10X_scRNA-seq,/seq/illumina/runs/41/41865/cellranger/cellran...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,NaN
130,Spine,41865_SB_200532_11018323,17,lumbar,HDBR15948,10X_scRNA-seq,/seq/illumina/runs/41/41865/cellranger/cellran...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,/lustre/scratch126/cellgen/team298/sm54/Data_I...,NaN


In [9]:
arr = os.listdir()

sample_list= metadata["Run_ID"].tolist()

sample_list = [x for x in sample_list if str(x) != 'nan']


filter_data = [x for x in arr if
              any(y in x for y in sample_list)]

In [11]:
filter_data

['HCA_BN_F13109713_and_HCA_BN_F13101383',
 'HCA_BN_F12863835_and_HCA_BN_F12865000',
 'HCA_BN_F12808062_and_HCA_BN_F12803238',
 'HCA_BN_F12482359_and_HCA_BN_F12477271',
 'HCA_BRA_F13774555_and_HCA_BRA_F13852282',
 'HCA_BN_F13177436_and_HCA_BN_F13177209',
 'HCA_BN_F12482364_and_HCA_BN_F12477276',
 'HCA_BN_F13177435_and_HCA_BN_F13177208',
 'HCA_BN_F12874046_and_HCA_BN_F12865678',
 'HCA_BN_F12482365_and_HCA_BN_F12477277',
 'HCA_BN_F13109711_and_HCA_BN_F13101381',
 'HCA_BN_F12874040_and_HCA_BN_F12865672',
 'HCA_BN_F13109712_and_HCA_BN_F13101382',
 'HCA_BRA_F13168992_and_HCA_BRA_F13168608',
 'HCA_BN_F12874042_and_HCA_BN_F12865674',
 'HCA_BN_F12874043_and_HCA_BN_F12865675',
 'HCA_BRA_F13168988_and_HCA_BRA_F13168604',
 'HCA_BN_F12482366_and_HCA_BN_F12477278',
 'HCA_BRA_F13774556_and_HCA_BRA_F13852283',
 'HCA_BRA_F13168989_and_HCA_BRA_F13168605',
 'HCA_BN_F12966456_and_HCA_BN_F12951832',
 'HCA_BN_F12808059_and_HCA_BN_F12803235',
 'HCA_BN_F12922483_and_HCA_BN_F12918721',
 'HCA_BN_F12922481_and_H

# Skull and meninges

In [12]:
len(filter_data)

66

In [13]:
filter_data.remove('HCA_BRA_F13774552_and_HCA_BRA_F13768422')

In [14]:
len(filter_data)

65

In [15]:
import warnings
warnings.filterwarnings('ignore')

adata, ad_list = read_10x_output(
    smp_list=filter_data,
    
    type='filtered'
)

In [16]:
adata

AnnData object with n_obs × n_vars = 924995 × 36601
    obs: 'sample_id', 'barcode'
    var: 'SYMBOL', 'ENSEMBL'

In [17]:
adata.obs["sample_id"].value_counts()[:10]

sample_id
HCA_BN_F12863835_and_HCA_BN_F12865000    34958
HCA_BN_F13109713_and_HCA_BN_F13101383    32474
HCA_BN_F13109710_and_HCA_BN_F13101380    22183
HCA_BN_F13109711_and_HCA_BN_F13101381    21755
HCA_BN_F12966457_and_HCA_BN_F12951833    21730
HCA_BN_F13177437_and_HCA_BN_F13177210    21195
HCA_BN_F12966460_and_HCA_BN_F12951836    20866
HCA_BN_F12966455_and_HCA_BN_F12951831    20620
HCA_BN_F12922487_and_HCA_BN_F12918725    20472
HCA_BN_F12966459_and_HCA_BN_F12951835    20320
Name: count, dtype: int64

In [28]:
adata.var.drop(columns=['ENSEMBL'], inplace=True)

In [29]:
adata

AnnData object with n_obs × n_vars = 924995 × 36601
    obs: 'sample_id', 'barcode'
    var: 'SYMBOL'

In [19]:
meninges_sample= sc.read_10x_h5('HCA_BRA_F13774552_and_HCA_BRA_F13768422/HCA_BRA_F13774552_and_HCA_BRA_F13768422_filtered.h5')

In [21]:
meninges_sample.var['ENSEMBL'] = meninges_sample.var.index
meninges_sample.var.rename(columns = {'gene_ids':'SYMBOL'}, inplace = True)
meninges_sample.var.index = meninges_sample.var['SYMBOL']
meninges_sample.var_names_make_unique()
meninges_sample.obs['sample_id'] = 'HCA_BRA_F13774552_and_HCA_BRA_F13768422'
meninges_sample.obs['barcode'] = meninges_sample.obs_names
meninges_sample.obs_names = meninges_sample.obs['barcode']+"_"+meninges_sample.obs['sample_id']


In [24]:
meninges_sample.var.drop(columns=['ENSEMBL'], inplace=True)

In [30]:
meninges_sample

AnnData object with n_obs × n_vars = 13587 × 36601
    obs: 'sample_id', 'barcode'
    var: 'SYMBOL'

In [34]:
meninges_skull=anndata.concat([adata,meninges_sample])

In [35]:
meninges_skull

AnnData object with n_obs × n_vars = 938582 × 36601
    obs: 'sample_id', 'barcode'

## save object 

In [38]:
meninges_skull.write_h5ad("/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object/data/meninges_skull_cellbender_filtered_raw_counts_20231018.h5ad")

# Spine

In [2]:
cd /lustre/scratch126/cellgen/team298/sm54/Data_Integration/Spine/data/cellranger_outputs_all_samples/

/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Spine/data/cellranger_outputs_all_samples


# Metadata

In [3]:
metadata= pd.read_csv("/home/jovyan/mount/gdrive/Spine/metadata_spine_DRteam.csv")

In [4]:
metadata

,PCW,region,subregion,dissociation,batch,irods_path
0,7.0,lumbar,cervical,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841321_G...
1,7.0,thoracic,thoracic,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841322_G...
2,7.0,cervical,lumbar,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841323_G...
3,5.0,cervical,cervical,no_trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621989_G...
4,5.0,cervical,cervical,trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621992_G...
...,...,...,...,...,...,...
63,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
metadata=metadata.dropna(how='all')

In [6]:
metadata

,PCW,region,subregion,dissociation,batch,irods_path
0,7.0,lumbar,cervical,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841321_G...
1,7.0,thoracic,thoracic,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841322_G...
2,7.0,cervical,lumbar,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841323_G...
3,5.0,cervical,cervical,no_trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621989_G...
4,5.0,cervical,cervical,trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621992_G...
...,...,...,...,...,...,...
57,17.0,cervical,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018320_G...
58,17.0,thoracic,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018321_G...
59,17.0,thoracic,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018322_G...
60,17.0,lumbar,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018323_G...


In [7]:
metadata["irods_path"]= metadata["irods_path"].astype(str)

/tmp/ipykernel_6039/933390425.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["irods_path"]= metadata["irods_path"].astype(str)


In [8]:
metadata

,PCW,region,subregion,dissociation,batch,irods_path
0,7.0,lumbar,cervical,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841321_G...
1,7.0,thoracic,thoracic,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841322_G...
2,7.0,cervical,lumbar,no_trypsin,HDBR15918,cellranger600_count_40813_SB_200532_10841323_G...
3,5.0,cervical,cervical,no_trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621989_G...
4,5.0,cervical,cervical,trypsin,HDBR15868,cellranger600_count_41025_SB_200532_10621992_G...
...,...,...,...,...,...,...
57,17.0,cervical,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018320_G...
58,17.0,thoracic,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018321_G...
59,17.0,thoracic,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018322_G...
60,17.0,lumbar,NaN,trypsin,HDBR15948,cellranger600_count_41865_SB_200532_11018323_G...


In [9]:
arr = os.listdir()

sample_list= metadata["irods_path"].tolist()

sample_list = [x for x in sample_list if str(x) != 'nan']


filter_data = [x for x in arr if
              any(y in x for y in sample_list)]

In [10]:
filter_data

['cellranger600_count_39102_SB_200532_10297929_GRCh38-2020-A',
 'cellranger600_count_42152_SB_200532_11254458_GRCh38-2020-A',
 'cellranger600_count_41025_SB_200532_10621989_GRCh38-2020-A',
 'cellranger600_count_39102_SB_200532_10297927_GRCh38-2020-A',
 'cellranger600_count_42152_SB_200532_11254464_GRCh38-2020-A',
 'cellranger600_count_41456_SB_200532_10863985_GRCh38-2020-A',
 'cellranger600_count_42152_SB_200532_11254461_GRCh38-2020-A',
 'cellranger600_count_41456_SB_200532_10863987_GRCh38-2020-A',
 'cellranger600_count_41865_SB_200532_11018320_GRCh38-2020-A',
 'cellranger600_count_40813_SB_200532_10841321_GRCh38-2020-A',
 'cellranger600_count_41865_SB_200532_11018319_GRCh38-2020-A',
 'cellranger600_count_39102_SB_200532_10297930_GRCh38-2020-A',
 'cellranger600_count_42152_SB_200532_11254463_GRCh38-2020-A',
 'cellranger600_count_42152_SB_200532_11254465_GRCh38-2020-A',
 'cellranger600_count_41456_SB_200532_10863988_GRCh38-2020-A',
 'cellranger600_count_41456_SB_200532_10863995_GRCh38-2

In [11]:
len(filter_data)

61

In [12]:
gc.collect()

0

In [13]:
sample.var

NameError: name 'sample' is not defined

In [14]:

def read_10x_output_spine(smp_list, type = 'filtered'):
    import os
    
    #Writing output from separate samples, processed using CellRanger, into a dictionary of Scanpy objects:
    ad = {}

    #Generate AnnData for each sample
    for sample_name in smp_list:
        path = sample_name
        for i in os.listdir(path):
            if type in i and 'h5' in i:
                file = i
        ad[sample_name] = sc.read_10x_h5(sample_name +'/'+file)
        ad[sample_name].var['SYMBOL'] = ad[sample_name].var.index
        #ad[sample_name].var.drop(columns=['gene_ids',"feature_types","genome"], inplace=True)
        ad[sample_name].var_names_make_unique() 
        ad[sample_name].obs['sample_id'] = sample_name
        ad[sample_name].obs['barcode'] = ad[sample_name].obs_names

    #Merge AnnData objects from all the samples together    
    from scipy.sparse import vstack
    stack = vstack([ad[x].X for x in smp_list]) # stack data
    adata = sc.AnnData(stack, var = ad[smp_list[0]].var)
    adata.obs = pd.concat([ad[x].obs for x in smp_list], axis = 0)

    for c in adata.obs.columns:
        adata.obs[c] = adata.obs[c].astype('str')
    adata.obs =  adata.obs.copy()

    return adata, ad

In [16]:
warnings.filterwarnings('ignore')

spine, ad_list = read_10x_output_spine(
    smp_list=filter_data,
    
    type='filtered'
)

In [19]:
spine

AnnData object with n_obs × n_vars = 547321 × 36601
    obs: 'sample_id', 'barcode'
    var: 'gene_ids', 'feature_types', 'genome', 'SYMBOL'

In [40]:
spine.var

,SYMBOL
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC141272.1,AC141272.1
AC023491.2,AC023491.2
AC007325.1,AC007325.1
AC007325.4,AC007325.4


In [21]:
spine.var.drop(columns=['gene_ids',"feature_types","genome"], inplace=True)

In [23]:
spine.var

,SYMBOL
MIR1302-2HG,MIR1302-2HG
FAM138A,FAM138A
OR4F5,OR4F5
AL627309.1,AL627309.1
AL627309.3,AL627309.3
...,...
AC141272.1,AC141272.1
AC023491.2,AC023491.2
AC007325.1,AC007325.1
AC007325.4,AC007325.4


In [26]:
spine.write_h5ad("/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object/data/spine_no_cellbender_filtered_raw_counts_20231018.h5ad")

# Combined anndata 

In [24]:
meninges_skull= sc.read_h5ad("/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object/data/meninges_skull_cellbender_filtered_raw_counts_20231018.h5ad")

In [33]:
meninges_skull.var["SYMBOL"]= meninges_skull.var.index

In [34]:
meninges_skull

AnnData object with n_obs × n_vars = 938582 × 36601
    obs: 'sample_id', 'barcode'
    var: 'SYMBOL'

In [31]:
spine

AnnData object with n_obs × n_vars = 547321 × 36601
    obs: 'sample_id', 'barcode'
    var: 'SYMBOL'

In [35]:
combined_object= anndata.concat([meninges_skull,spine])

In [38]:
combined_object.obs

,sample_id,barcode
TAAGCCTAGGGACCTC-1_HCA_BN_F13109713_and_HCA_BN_F13101383,HCA_BN_F13109713_and_HCA_BN_F13101383,TAAGCCTAGGGACCTC-1
CGCACAATCCAGGTTG-1_HCA_BN_F13109713_and_HCA_BN_F13101383,HCA_BN_F13109713_and_HCA_BN_F13101383,CGCACAATCCAGGTTG-1
CGCTTCTAGGATTGCT-1_HCA_BN_F13109713_and_HCA_BN_F13101383,HCA_BN_F13109713_and_HCA_BN_F13101383,CGCTTCTAGGATTGCT-1
CTTTAGTTCCCAGTAG-1_HCA_BN_F13109713_and_HCA_BN_F13101383,HCA_BN_F13109713_and_HCA_BN_F13101383,CTTTAGTTCCCAGTAG-1
CTGACCAAGGCTGTGC-1_HCA_BN_F13109713_and_HCA_BN_F13101383,HCA_BN_F13109713_and_HCA_BN_F13101383,CTGACCAAGGCTGTGC-1
...,...,...
TTTGGTTCACTACACA-1,cellranger600_count_40813_SB_200532_10841316_G...,TTTGGTTCACTACACA-1
TTTGGTTTCAGGGTAG-1,cellranger600_count_40813_SB_200532_10841316_G...,TTTGGTTTCAGGGTAG-1
TTTGGTTTCCGCAACG-1,cellranger600_count_40813_SB_200532_10841316_G...,TTTGGTTTCCGCAACG-1
TTTGTTGGTAACGTTC-1,cellranger600_count_40813_SB_200532_10841316_G...,TTTGTTGGTAACGTTC-1


In [41]:
del spine

In [42]:
del meninges_skull

In [43]:
gc.collect()

5696

In [ ]:
combined_object.write_h5ad("/lustre/scratch126/cellgen/team298/sm54/Data_Integration/Integrated_Object/data/combined_spine_skull_meninges_raw_counts_20231018.h5ad")

In [47]:
combined_object

AnnData object with n_obs × n_vars = 1485903 × 36601
    obs: 'sample_id', 'barcode'